In [ ]:
%reload_ext autoreload
%autoreload 2
    
import sys
import os
repo_dir = '/srv/local/work/swang141/Sheng_repo/'
nlp_tool_dir = '/srv/local/work/swang141/PatientSetAnnotation/Sheng/analysis/stanford_parser_java/stanford-parser-full-2017-06-09/'
pubmed_dir = '/srv/local/work/swang141/PatientSetAnnotation/Sheng/data/pubmed/'
sys.path.append(repo_dir)
os.chdir(repo_dir)

from pikachu.datasets.WordNet import WordNet
from pikachu.datasets.ImprovedWordNet import ImprovedWordNet
from pikachu.utils.nlp import parse_word_net
from pikachu.models.generate_sentence.extractive_gen import ExtractGenSent
from pikachu.datasets.SubGraph import SubGraph
import operator
import collections

Net_obj = WordNet()
pubmed_word_net = ['data/Pubmed/word_network/predict_abst_180717','data/Pubmed/word_network/all_abst_180305']
Net_obj.ReadWordNet(pubmed_word_net,verbal=True,min_freq_cutoff=0)

Net_obj.ReadWordType()

stop_word_file = 'data/NLP_Dictionary/stopwords.txt'
#stop_word_list = parse_word_net.GenerateStopWordList(Net_obj.word_ct,Net_obj.edge_ct,stop_word_file,Net_obj.word_type)
stop_word_list = parse_word_net.GetStopWordList(stop_word_file)
stop_word_list_manually = parse_word_net.GetStopWordList('data/NLP_Dictionary/stopwords_manually.txt')
stop_word_list = stop_word_list.union(stop_word_list_manually)

Net_obj.ReadEdgeType(stop_word_list)

candidate_gene,monarch_d2g = Net_obj.ReadNodeTypeMonarchDiseaseGene()
selected_kg_l = [Net_obj.literome_g2g,Net_obj.synonym_g2g,Net_obj.hpo_d2d,Net_obj.go_f2f]

ImproveNet_obj = ImprovedWordNet(Net_obj,selected_kg_l)

#result_dir = '/srv/local/work/swang141/PatientSetAnnotation/Sheng/result/network_flow/cancer/'

gene_score = collections.defaultdict(dict)
npath = collections.defaultdict(dict)
ImproveNet_obj.reload()



finished 0 1 mitotic nuclear division regulates
finished 30000000 6 triglyceride transport octapressin
finished 60000000 2 response to cortisone acupuncture therapy
finished 0 7746784 was for
finished 30000000 8 mitochondrial genome idiopathic


In [2]:
import numpy as np
from pikachu.utils.evaluate.evaluate import evaluate_vec

max_ngh = 200
max_path_L = 4
edge_wt_thres = 1
max_dup_edge_type = 2
result_dir = '/srv/local/work/swang141/Sheng_repo/result/network_flow/monarch/'

for ci,s in enumerate(monarch_d2g):
    if s=='all':
        continue
    print s,len(monarch_d2g[s])
    predict = []
    truth = []
    for i,t in enumerate(candidate_gene):
        if t not in monarch_d2g[s]:
            a = np.random.rand(1)
            if a>0.01:
                continue
        if not ImproveNet_obj.ValidPath(s,t):
            continue
        image_file = result_dir + s +'_' + t
        for max_ngh in range(1,10,1):            
            sub_graph = SubGraph(ImproveNet_obj,s,set([t]), max_path_L = max_path_L,edge_wt_thres = edge_wt_thres,max_ngh=max_ngh)
            gene_score[s][t],npath[s][t],edge_list,node_list = sub_graph.CalSubGraphScore(image_file = image_file, max_dup_edge_type = max_dup_edge_type,dfs_max_depth=max_path_L+1)
            if gene_score[s][t] > -1 and npath[s][t]>1:
                print s,t,gene_score[s][t],npath[s][t],t in monarch_d2g[s],max_ngh
                #SenGene_obj.get_graph_sentence(edge_list,image_file,Net_obj)
                break
        print i,len(candidate_gene),t in monarch_d2g[s]
        predict.append(gene_score[s][t])
        truth.append(int(t in monarch_d2g[s]))
        auc_pred = evaluate_vec(predict,truth)[0]
        print s, auc_pred

NameError: name 'monarch_d2g' is not defined

In [ ]:
np.random.rand(1)